In [ ]:
# 📊 Chargement des données depuis MongoDB
from lib.db import read_mongodb_to_dataframe

print("🔄 Connexion à MongoDB et chargement des données...")

# Chargement des propriétés vendues
properties_db = read_mongodb_to_dataframe(
    db='real_estate_db', 
    collection='properties', 
    query={'vendue': True},  # Filtre pour les propriétés vendues uniquement
    no_id=False  # Garder l'ID pour référence
)

print(f"✅ Données chargées avec succès!")
print(f"📊 Nombre de propriétés: {len(properties_db):,}")
print(f"📋 Nombre de colonnes: {len(properties_db.columns)}")


In [ ]:
# 📦 Import des classes modulaires
from lib.property_analysis import (
    PropertyAnalyzer, 
    PropertyDataProcessor, 
    PropertyClassifier, 
    FeatureSelector
)

print("📦 Classes modulaires importées avec succès!")

# Configuration du processeur de données
data_processor = PropertyDataProcessor(
    missing_threshold=0.05  # Supprimer les colonnes avec >95% de valeurs manquantes
)

# Configuration du classificateur de propriétés
property_classifier = PropertyClassifier()

# Configuration du sélecteur de variables
feature_selector = FeatureSelector(
    cv_folds=5,              # Validation croisée 5-folds
    random_state=42,         # Reproductibilité
    max_iter=10000,          # Convergence Lasso
    tolerance=1e-3,          # Tolérance de convergence
    rf_n_estimators=100,     # Nombre d'arbres Random Forest
    rf_threshold=0.01        # Seuil d'importance des variables
)

# Création de l'analyseur principal avec injection de dépendances
analyzer = PropertyAnalyzer(
    data_processor=data_processor,
    property_classifier=property_classifier,
    feature_selector=feature_selector
)

print("✅ Analyseur configuré et prêt!")


In [ ]:
# 🚀 Exécution du pipeline complet d'analyse
print("🔄 Lancement de l'analyse automatisée en 5 étapes...")

try:
    # Exécution de l'analyse complète avec la fonction corrigée
    results = analyzer.analyze_properties(properties_db, target_column='price')
    
    print(f"\n🎉 === PIPELINE TERMINÉ AVEC SUCCÈS ===")
    print(f"✅ Toutes les 5 étapes ont été exécutées:")
    print(f"   1. 🧹 Nettoyage des données")
    print(f"   2. 🏷️ Classification des propriétés")  
    print(f"   3. 🔧 Préparation des données")
    print(f"   4. 🎯 Sélection des variables")
    print(f"   5. 📊 Analyse des résultats")
    
except Exception as e:
    print(f"❌ Erreur durante l'analyse: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# 🎁 Variables disponibles pour les étapes suivantes
if 'results' in locals():
    X = analyzer.processed_data['X']
    y = analyzer.processed_data['y']
    df_classified = analyzer.processed_data['df_classified']
    df_prepared = analyzer.processed_data['df_prepared']
    important_features = results['selected_features']
    
    print(f"📋 === RÉSULTATS DE L'ANALYSE ===")
    
    # Forme des données
    print(f"\n🏠 Transformation des données:")
    print(f"   📊 Originales: {results['shape_original']}")
    print(f"   📊 Traitées: {results['shape_processed']}")
    
    # Classification des propriétés
    print(f"\n🏷️ Classification des propriétés:")
    for category, count in results['classification_stats']['counts'].items():
        pct = results['classification_stats']['percentages'][category]
        print(f"   🏠 {category}: {count:,} propriétés ({pct:.1f}%)")
    
    # Variables sélectionnées
    print(f"\n🎯 Variables sélectionnées:")
    print(f"   📈 Total: {len(results['selected_features'])} variables")
    print(f"   📝 Variables: {results['selected_features'][:10]}...")  # Afficher les 10 premières
    
    # Importance des variables
    if results['feature_importance']:
        print(f"\n🌟 Top 5 des variables les plus importantes:")
        sorted_importance = sorted(results['feature_importance'].items(), key=lambda x: x[1], reverse=True)
        for i, (feature, importance) in enumerate(sorted_importance[:5], 1):
            print(f"   {i}. {feature}: {importance:.4f}")
    
    # Résumé final
    summary = analyzer.get_summary()
    print(f"\n📈 === RÉSUMÉ FINAL ===")
    print(f"   🏠 Total propriétés: {summary['total_properties']:,}")
    print(f"   📊 Features totales: {summary['total_features']}")
    print(f"   🎯 Features sélectionnées: {summary['selected_features_count']}")
    print(f"   📉 Réduction: {summary['reduction_percentage']:.1f}%")
    print(f"   💰 Prix moyen: {summary['price_stats']['mean']:,.2f}$")
    
    print(f"\n🔧 Variables disponibles pour la suite:")
    print(f"   📊 X: Matrice des features ({X.shape})")
    print(f"   🎯 y: Vecteur cible ({y.shape})")
    print(f"   🏠 df_classified: DataFrame avec classification ({df_classified.shape})")
    print(f"   🛠️  df_prepared: DataFrame préparé pour ML ({df_prepared.shape})")
    print(f"   📝 important_features: Liste des variables importantes")
else:
    print("❌ Aucun résultat disponible - L'analyse a probablement échoué")


In [ ]:
import pandas as pd
import numpy as np
from lib import PropertyAnalyzer, MongoDBLoader
import warnings
warnings.filterwarnings('ignore')

# Option 1: Avec données MongoDB (recommandé)
# loader = MongoDBLoader()
# property_types = loader.load_property_types()
# analyzer = PropertyAnalyzer(property_types_data=property_types)

# Option 2: Avec données d'exemple pour la normalisation des types
property_types_example = [
    {
        "_id": "maison",
        "display_names": {"fr": "Maison", "en": "House"},
        "category": "Résidentiel",
        "typical_characteristics": ["living_area", "lot_size", "bedrooms", "bathrooms"]
    },
    {
        "_id": "condo",
        "display_names": {"fr": "Condo", "en": "Condominium"},
        "category": "Résidentiel",
        "typical_characteristics": ["living_area", "bedrooms", "bathrooms"]
    },
    {
        "_id": "duplex",
        "display_names": {"fr": "Duplex", "en": "Duplex"},
        "category": "Résidentiel",
        "typical_characteristics": ["living_area", "lot_size", "bedrooms", "bathrooms"]
    }
]

# Option 3: Sans données de types (utilise la normalisation basique)
# analyzer = PropertyAnalyzer()

# Initialiser l'analyseur avec les données de types
analyzer = PropertyAnalyzer(property_types_data=property_types_example)

print("🏠 Analyseur immobilier initialisé avec succès!")
print("🔄 Normalisateur de types de propriétés configuré!")


In [ ]:
# Option 1: Charger depuis MongoDB
# loader = MongoDBLoader()
# df = loader.load_properties()
# property_types = loader.load_property_types()

# Option 2: Charger depuis un fichier CSV/Excel
# df = pd.read_csv('votre_fichier.csv')

# Option 3: Utiliser des données d'exemple (pour test)
np.random.seed(42)
n_samples = 1000

df = pd.DataFrame({
    'price': np.random.normal(500000, 150000, n_samples),
    'living_area': np.random.normal(1500, 400, n_samples),
    'lot_size': np.random.normal(5000, 2000, n_samples),
    'bedrooms': np.random.randint(1, 6, n_samples),
    'bathrooms': np.random.randint(1, 4, n_samples),
    'year_built': np.random.randint(1950, 2024, n_samples),
    'type': np.random.choice(['maison', 'condo', 'duplex'], n_samples),
    'city': np.random.choice(['Montreal', 'Quebec', 'Laval', 'Gatineau'], n_samples),
    'region': np.random.choice(['Montreal', 'Quebec', 'Outaouais'], n_samples),
    'building_style': np.random.choice(['contemporain', 'traditionnel', 'moderne'], n_samples),
    'municipal_evaluation_total': np.random.normal(450000, 120000, n_samples),
    # Colonnes à supprimer (simulation données MongoDB)
    '_id': range(n_samples),
    'link': ['http://example.com'] * n_samples,
    'images': [['img1.jpg', 'img2.jpg']] * n_samples,
    'extraction_metadata': [{'date': '2024-01-01'}] * n_samples
})

# Ajouter quelques valeurs manquantes
df.loc[df.sample(50).index, 'lot_size'] = np.nan
df.loc[df.sample(30).index, 'year_built'] = np.nan

print(f"📊 Données chargées: {df.shape}")
print(f"📝 Colonnes: {list(df.columns)}")
df.head()


In [ ]:
# Valider et explorer les données
df_step1 = analyzer.validate_and_explore(df, target_column='price')

# Vous pouvez ajouter votre propre exploration ici
print("\n🔍 Exploration supplémentaire:")
print(f"Types de données:")
print(df_step1.dtypes)

print(f"\nValeurs manquantes par colonne:")
print(df_step1.isnull().sum())


In [ ]:
# Nettoyer les données
df_step2 = analyzer.clean_data(df_step1)

print("\n🔍 Vérification après nettoyage:")
print(f"Colonnes restantes: {list(df_step2.columns)}")
print(f"Forme des données: {df_step2.shape}")

df_step2.head()


In [ ]:
# Normaliser les variables catégorielles
df_step3 = analyzer.normalize_variables(df_step2)

print("\n🔍 Vérification après normalisation:")
print(f"Colonnes: {list(df_step3.columns)}")

# Vérifier les changements dans les variables catégorielles
categorical_cols = df_step3.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n📊 Variable '{col}':")
    print(f"   Valeurs uniques: {df_step3[col].nunique()}")
    print(f"   Valeurs: {list(df_step3[col].unique())}")

df_step3.head()


In [ ]:
# Encoder les variables catégorielles
df_step4 = analyzer.encode_features(df_step3)

print("\n🔍 Vérification après encodage:")
print(f"Nouvelles colonnes: {list(df_step4.columns)}")
print(f"Types de données après encodage:")
print(df_step4.dtypes)

df_step4.head()


In [ ]:
# Imputer les valeurs manquantes
df_step5 = analyzer.impute_missing_values(df_step4)

print("\n🔍 Vérification après imputation:")
print(f"Valeurs manquantes restantes: {df_step5.isnull().sum().sum()}")
print(f"Statistiques des colonnes imputées:")
print(df_step5.describe())

df_step5.head()


In [ ]:
# Classifier les propriétés
df_step6 = analyzer.classify_properties(df_step5)

print("\n🔍 Vérification de la classification:")
print(f"Nouvelle colonne: 'classification_immobiliere'")
print(f"Distribution des classifications:")
print(df_step6['classification_immobiliere'].value_counts())

# Visualiser la distribution des prix par classification
print("\n💰 Prix moyen par catégorie:")
price_by_class = df_step6.groupby('classification_immobiliere')['price'].agg(['mean', 'median', 'count'])
print(price_by_class)

df_step6.head()


In [ ]:
# Préparer les données pour la modélisation
X, y = analyzer.prepare_for_modeling(df_step6, target_column='price')

print("\n🔍 Vérification des données préparées:")
print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nColonnes features:")
for i, col in enumerate(X.columns, 1):
    print(f"  {i}. {col}")

print(f"\nStatistiques de la variable cible:")
print(y.describe())

X.head()


In [ ]:
# Sélectionner les variables importantes
selected_features = analyzer.select_features(X, y)

print("\n🔍 Analyse des variables sélectionnées:")
print(f"Variables sélectionnées: {selected_features}")

# Créer un sous-ensemble avec seulement les variables sélectionnées
X_selected = X[selected_features]
print(f"\nNouveaux features (X_selected): {X_selected.shape}")

X_selected.head()


In [ ]:
# Sélectionner les variables par type de propriété
classification_features = analyzer.select_features_by_classification(
    X, y, df_step6['classification_immobiliere']
)

print("\n🔍 Analyse des variables par type:")
if classification_features:
    for prop_type, features in classification_features.items():
        print(f"\n🏷️ {prop_type}:")
        print(f"   Variables importantes: {features}")
        
        # Créer un sous-ensemble pour ce type
        mask = df_step6['classification_immobiliere'] == prop_type
        X_type = X[mask][features]
        y_type = y[mask]
        print(f"   Données pour ce type: X={X_type.shape}, y={y_type.shape}")
else:
    print("Pas de sélection spécifique par type disponible")


In [ ]:
# Calculer l'importance des variables
feature_importance = analyzer.calculate_feature_importance(X, y)

print("\n🔍 Analyse détaillée de l'importance:")
importance_df = pd.DataFrame([
    {'feature': feature, 'importance': importance}
    for feature, importance in feature_importance.items()
]).sort_values('importance', ascending=False)

print("\n📊 Tableau d'importance complet:")
print(importance_df)

# Visualisation simple
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
top_features = importance_df.head(10)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title('Top 10 Variables les Plus Importantes')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Générer le résumé final
final_results = analyzer.final_summary()

print("\n📋 Résultats structurés:")
print(f"Résultats disponibles: {final_results.keys()}")

# Créer un DataFrame de résumé
summary_data = {
    'Métrique': [
        'Lignes originales',
        'Lignes après traitement',
        'Colonnes originales',
        'Colonnes après traitement',
        'Variables sélectionnées',
        'Taux de conservation des variables'
    ],
    'Valeur': [
        final_results.get('shape_original', (0, 0))[0],
        final_results.get('shape_processed', (0, 0))[0],
        final_results.get('shape_original', (0, 0))[1],
        final_results.get('shape_processed', (0, 0))[1],
        len(final_results.get('selected_features', [])),
        f"{(len(final_results.get('selected_features', [])) / (final_results.get('shape_processed', (0, 1))[1] - 1) * 100):.1f}%"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n📊 Tableau de résumé:")
print(summary_df.to_string(index=False))


In [ ]:
# Maintenant vous avez toutes les données prêtes pour la modélisation
print("🎯 Données prêtes pour la modélisation:")
print(f"\n📊 X (features complètes): {X.shape}")
print(f"📊 X_selected (features sélectionnées): {X_selected.shape}")
print(f"🎯 y (target): {y.shape}")
print(f"🏠 df_step6 (données complètes avec classification): {df_step6.shape}")

print(f"\n✅ Variables sélectionnées pour la modélisation:")
for i, feature in enumerate(selected_features, 1):
    print(f"  {i}. {feature}")

print(f"\n🏷️ Classifications disponibles:")
for category, stats in final_results.get('classification_stats', {}).get('counts', {}).items():
    print(f"  • {category}: {stats} propriétés")

print("\n🎉 Prêt pour l'entraînement de modèles!")
